In [36]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import pandas as pd

In [54]:
def get_jobs(keyword, num_jobs, verbose, path, timesleep):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(timesleep)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CLASS_NAME, "react-job-listing").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element(By.CLASS_NAME, "modal_closeIcon-svg").click()  #clicking to the X.
            print('X worked')
        except NoSuchElementException:
            print('X failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_element(By.CLASS_NAME, "jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
                "Salary Estimate" : salary_estimate,
                "Job Description" : job_description,
                "Rating" : rating,
                "Company Name" : company_name,
                "Location" : location,
                "Headquarters" : headquarters,
                "Size" : size,
                "Founded" : founded,
                "Type of ownership" : type_of_ownership,
                "Industry" : industry,
                "Sector" : sector,
                "Revenue" : revenue,
                "Competitors" : competitors})
                #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [55]:
path = r'C:\Users\tangu\anaconda3\chromedriver'
df = get_jobs('data scientist', 15, False, path, 5)
df

C:\Users\tangu\AppData\Local\Temp\ipykernel_928\142447496.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


X worked


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".jl"}
  (Session info: chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A278B3+2193587]
	Ordinal0 [0x009C0681+1771137]
	Ordinal0 [0x008D41A8+803240]
	Ordinal0 [0x009024A0+992416]
	Ordinal0 [0x0090273B+993083]
	Ordinal0 [0x0092F7C2+1177538]
	Ordinal0 [0x0091D7F4+1103860]
	Ordinal0 [0x0092DAE2+1170146]
	Ordinal0 [0x0091D5C6+1103302]
	Ordinal0 [0x008F77E0+948192]
	Ordinal0 [0x008F86E6+952038]
	GetHandleVerifier [0x00CD0CB2+2738370]
	GetHandleVerifier [0x00CC21B8+2678216]
	GetHandleVerifier [0x00AB17AA+512954]
	GetHandleVerifier [0x00AB0856+509030]
	Ordinal0 [0x009C743B+1799227]
	Ordinal0 [0x009CBB68+1817448]
	Ordinal0 [0x009CBC55+1817685]
	Ordinal0 [0x009D5230+1856048]
	BaseThreadInitThunk [0x7707FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77247A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77247A6E+238]


In [ ]:
<li class="react-job-listing css-bkasv9 eigr9kq0" data-brandviews="BRAND:n=jsearch-job-listing:eid=19080:jlid=1008047913702" data-id="1008047913702" data-adv-type="GENERAL" data-is-organic-job="false" data-ad-order-id="1110586" data-sgoc-id="1019" data-is-easy-apply="false" data-normalize-job-title="Data Scientist" data-job-loc="Livermore, CA" data-job-loc-id="1147367" data-job-loc-type="C" data-selected="true" data-test="jobListing" style="" data-triggered-brandview=""><div class="d-flex flex-column job-search-key-1pzmdmc e1rrn5ka1"><a href="/partner/jobListing.htm?pos=101&amp;ao=1110586&amp;s=58&amp;guid=00000182a3dac3b4b32cdf03ff55f25a&amp;src=GD_JOB_AD&amp;t=SR&amp;vt=w&amp;cs=1_b75ffeeb&amp;cb=1660606399845&amp;jobListingId=1008047913702&amp;cpc=D2A6DBF304636DC4&amp;jrtk=3-0-1gahtlgv2joqo801-1gahtlgvk2bek000-ee6c1ea70fbe40cb--6NYlbfkN0BdI5vrJcl4iNCACeycF7SOUtI3c7RKoL2EvjheEbCbdbbBOMBjX1TjRNHGhGl97pANXQ9knR4UeMlLHlrpc1Vr360v06GC93L9DUl8duBbhx3fbdzqaTGdcr3X2Hon2QdUL7meBaX-vM4nm_-KGnswbs60OUVN0ue1EHvtdSAXBSpJlJYPlxRgFc_CNv-kYOhmsSLsFFRJ3ysQPe58qRGZRm2km8imqc_fm325RSfYKuZcA_-rY3KobBVGHLqikMGAiIStXCmU1qqhHVKewnig1VgIlPvTvfu_bE2ioOmMRZp_-pEKlfz4KWidN-ksDLMiQfRcjKe32dZG-Lch4WRrtthLmDKU0eDhquMJO_Lgvq-aE-wFpJhCf9mKuEskdE8rzmBlZBLDP1MY49HO7a5uGoSu1u5lJwCEYQnyGPqkMQ82gjFjKvMOd9VgYlpBkKbU3NMv6LLzCZNCOvGKdiSDOcfG8WrcwNnR3kQ9RXzmAbLyoLsq0UgCCg9tiZHY1NEK-Ug2-bRk32OYehEL7CuDwiEBTLlPsL-URVTtdAYJCQCb-NJ5UQYhxWn1sKfKjfXXI_UlrRPV9-Tm_ZtwQwFG3tLU9scxsIDLXp45F11hsE8OZgr5UAx8ntKq0PK6c78LPbELcgaOr4a7Pq90784AeWCkm0mPFH0FGvGzlyp37ubTwyzpNluLTvPRMMJAWVU%3D" rel="nofollow noopener noreferrer" target="_blank" class="jobLink" title="Sandia National Laboratories" style="pointer-events: all;"><span class=" job-search-key-9ujsbx euyrj9o0"><img src="https://media.glassdoor.com/sql/19080/sandia-national-laboratories-squarelogo-1582219467257.png" alt="Sandia National Laboratories Logo" title="Sandia National Laboratories Logo"></span></a><span class=" job-search-key-srfzj0 e1cjmv6j0">4.1<i class="job-search-key-0 e1cjmv6j1"></i></span></div><div class="d-flex flex-column pl-sm css-1buaf54 job-search-key-1mn3dn8 e1rrn5ka0"><div class="d-flex justify-content-between align-items-start"><a href="/partner/jobListing.htm?pos=101&amp;ao=1110586&amp;s=58&amp;guid=00000182a3dac3b4b32cdf03ff55f25a&amp;src=GD_JOB_AD&amp;t=SR&amp;vt=w&amp;cs=1_b75ffeeb&amp;cb=1660606399845&amp;jobListingId=1008047913702&amp;cpc=D2A6DBF304636DC4&amp;jrtk=3-0-1gahtlgv2joqo801-1gahtlgvk2bek000-ee6c1ea70fbe40cb--6NYlbfkN0BdI5vrJcl4iNCACeycF7SOUtI3c7RKoL2EvjheEbCbdbbBOMBjX1TjRNHGhGl97pANXQ9knR4UeMlLHlrpc1Vr360v06GC93L9DUl8duBbhx3fbdzqaTGdcr3X2Hon2QdUL7meBaX-vM4nm_-KGnswbs60OUVN0ue1EHvtdSAXBSpJlJYPlxRgFc_CNv-kYOhmsSLsFFRJ3ysQPe58qRGZRm2km8imqc_fm325RSfYKuZcA_-rY3KobBVGHLqikMGAiIStXCmU1qqhHVKewnig1VgIlPvTvfu_bE2ioOmMRZp_-pEKlfz4KWidN-ksDLMiQfRcjKe32dZG-Lch4WRrtthLmDKU0eDhquMJO_Lgvq-aE-wFpJhCf9mKuEskdE8rzmBlZBLDP1MY49HO7a5uGoSu1u5lJwCEYQnyGPqkMQ82gjFjKvMOd9VgYlpBkKbU3NMv6LLzCZNCOvGKdiSDOcfG8WrcwNnR3kQ9RXzmAbLyoLsq0UgCCg9tiZHY1NEK-Ug2-bRk32OYehEL7CuDwiEBTLlPsL-URVTtdAYJCQCb-NJ5UQYhxWn1sKfKjfXXI_UlrRPV9-Tm_ZtwQwFG3tLU9scxsIDLXp45F11hsE8OZgr5UAx8ntKq0PK6c78LPbELcgaOr4a7Pq90784AeWCkm0mPFH0FGvGzlyp37ubTwyzpNluLTvPRMMJAWVU%3D" rel="nofollow noopener noreferrer" target="_blank" class=" job-search-key-l2wjgv e1n63ojh0 jobLink" style="pointer-events: all;"><span>Sandia National Laboratories</span></a><div class="align-self-end d-flex flex-nowrap align-items-start"><span class="save-job-button-1008047913702 nowrap job-search-key-14vetv2 e4teh7x0" data-test="save-job"><span class="SVGInline css-9th5vf"><svg class="SVGInline-svg css-9th5vf-svg" style="width: 20px;height: 20px;" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24"><path d="M12 5.11l.66-.65a5.56 5.56 0 017.71.19 5.63 5.63 0 010 7.92L12 21l-8.37-8.43a5.63 5.63 0 010-7.92 5.56 5.56 0 017.71-.19zm7.66 6.75a4.6 4.6 0 00-6.49-6.51L12 6.53l-1.17-1.18a4.6 4.6 0 10-6.49 6.51L12 19.58z" fill="currentColor" fill-rule="evenodd"></path></svg></span></span></div></div><a class="css-h3hi0t jobLink css-1rd3saf eigr9kq1" data-test="job-link" href="/partner/jobListing.htm?pos=101&amp;ao=1110586&amp;s=58&amp;guid=00000182a3dac3b4b32cdf03ff55f25a&amp;src=GD_JOB_AD&amp;t=SR&amp;vt=w&amp;cs=1_b75ffeeb&amp;cb=1660606399845&amp;jobListingId=1008047913702&amp;cpc=D2A6DBF304636DC4&amp;jrtk=3-0-1gahtlgv2joqo801-1gahtlgvk2bek000-ee6c1ea70fbe40cb--6NYlbfkN0BdI5vrJcl4iNCACeycF7SOUtI3c7RKoL2EvjheEbCbdbbBOMBjX1TjRNHGhGl97pANXQ9knR4UeMlLHlrpc1Vr360v06GC93L9DUl8duBbhx3fbdzqaTGdcr3X2Hon2QdUL7meBaX-vM4nm_-KGnswbs60OUVN0ue1EHvtdSAXBSpJlJYPlxRgFc_CNv-kYOhmsSLsFFRJ3ysQPe58qRGZRm2km8imqc_fm325RSfYKuZcA_-rY3KobBVGHLqikMGAiIStXCmU1qqhHVKewnig1VgIlPvTvfu_bE2ioOmMRZp_-pEKlfz4KWidN-ksDLMiQfRcjKe32dZG-Lch4WRrtthLmDKU0eDhquMJO_Lgvq-aE-wFpJhCf9mKuEskdE8rzmBlZBLDP1MY49HO7a5uGoSu1u5lJwCEYQnyGPqkMQ82gjFjKvMOd9VgYlpBkKbU3NMv6LLzCZNCOvGKdiSDOcfG8WrcwNnR3kQ9RXzmAbLyoLsq0UgCCg9tiZHY1NEK-Ug2-bRk32OYehEL7CuDwiEBTLlPsL-URVTtdAYJCQCb-NJ5UQYhxWn1sKfKjfXXI_UlrRPV9-Tm_ZtwQwFG3tLU9scxsIDLXp45F11hsE8OZgr5UAx8ntKq0PK6c78LPbELcgaOr4a7Pq90784AeWCkm0mPFH0FGvGzlyp37ubTwyzpNluLTvPRMMJAWVU%3D" rel="nofollow noopener noreferrer" target="_blank" style="pointer-events: all;"><span>Information Security Data Scientist</span></a><div class="d-flex flex-wrap job-search-key-1m2z0go e1rrn5ka2"><span class="css-1buaf54 pr-xxsm job-search-key-iii9i8 e1rrn5ka4">Livermore, CA</span></div><div class="d-flex flex-wrap job-search-key-1a46cm1 e1rrn5ka3"><div class="css-1buaf54 pr-xxsm"><span data-test="detailSalary" class="job-search-key-1hbqxax e1wijj240">$91K - $131K <span class="job-search-key-0 e1wijj242">(Glassdoor est.)</span><span data-test="salaryIcon" class="SVGInline greyInfoIcon"><svg class="SVGInline-svg greyInfoIcon-svg" width="16" height="16" viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg"><g id="prefix__info-16-px" stroke="none" stroke-width="1" fill="none" fill-rule="evenodd"><path d="M8 14A6 6 0 118 2a6 6 0 010 12zm0-1A5 5 0 108 3a5 5 0 000 10zm-.6-5.6a.6.6 0 111.2 0V11a.6.6 0 01-1.2 0V7.4zM8 5.6a.6.6 0 110-1.2.6.6 0 010 1.2z" id="prefix__a" fill="#505863"></path></g></svg></span><div class="d-none"></div></span></div><div class="d-flex justify-content-between css-wzzvg6"><div class="d-flex flex-wrap-reverse css-13cn532 css-pa6dqi"><div class="mx-xxsm pr-std css-1ieon5s" data-test="viewedDate">Viewed on August 15</div><div class="d-flex flex-wrap"></div></div><div data-test="job-age" class="d-flex align-items-end pl-std css-17n8uzw">13d</div></div></div></div></li>
<svg class="SVGInline-svg modal_closeIcon-svg" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24"><path d="M13.34 12l5.38-5.38a.95.95 0 00-1.34-1.34L12 10.66 6.62 5.28a.95.95 0 00-1.34 1.34L10.66 12l-5.38 5.38a.95.95 0 001.34 1.34L12 13.34l5.38 5.38a.95.95 0 001.34-1.34z" fill="currentColor" fill-rule="evenodd"></path></svg>